<a href="https://colab.research.google.com/github/dh1105/Sentence-Entailment/blob/main/Sentence_Entailment_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install datasets transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00


In [8]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 6.5 MB/s eta 0:00:00


In [2]:
'''
0 (entailment)
1 (neutral)
2 (contradiction)
'''
from datasets import load_dataset
MNLI = load_dataset("multi_nli")
for split, dataset in MNLI.items():
    # print(split)
    dataset.to_csv(f"mnli-{split}.csv", index=None)

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/393 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

In [3]:
import pandas as pd
import re
import pickle
import os
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
# from torchmetrics.functional import precision_recall

import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer

from torch import nn
import torch.nn.functional as F
from tqdm import tqdm

from transformers import *

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `sr

In [ ]:
train_df = pd.read_csv('/content/mnli-train.csv', usecols = ['label', 'premise', 'hypothesis'])[:10000]
train_df = train_df.dropna()
train_df.head() # len = 392662

,premise,hypothesis,label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0
3,How do you know? All this is their information...,This information belongs to them.,0
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1


In [ ]:
len(train_df)

9998

In [ ]:
valid_m_df = pd.read_csv('/content/mnli-validation_matched.csv', usecols = ['label', 'premise', 'hypothesis'])
valid_m_df = valid_m_df.dropna()
valid_m_df.head() # len = 9815

,premise,hypothesis,label
0,The new rights are nice enough,Everyone really likes the newest benefits,1
1,This site includes a list of all award winners...,The Government Executive articles housed on th...,2
2,uh i don't know i i have mixed emotions about ...,"I like him for the most part, but would still ...",0
3,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...,2
4,i don't know um do you do a lot of camping,I know exactly.,2


In [ ]:
valid_mm_df = pd.read_csv('/content/mnli-validation_mismatched.csv', usecols = ['label', 'premise', 'hypothesis'])
valid_mm_df = valid_mm_df.dropna()
valid_mm_df.head() # len = 9832

,premise,hypothesis,label
0,Your contribution helped make it possible for ...,Your contributions were of no help with our st...,2
1,"The answer has nothing to do with their cause,...",Dictionaries are indeed exercises in bi-unique...,2
2,We serve a classic Tuscan meal that includes ...,We serve a meal of Florentine terrine.,0
3,"A few months ago, Carl Newton and I wrote a le...",Carl Newton and I have never had any other pre...,2
4,"I was on this earth you know, I've lived on th...",I don't yet know the reason why I have lived o...,0


In [ ]:
type(valid_mm_df['premise'][0])

str

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# tokenizer.encode('Hi, how are you?', add_special_tokens = False)

[7632, 1010, 2129, 2024, 2017, 1029]

In [ ]:
# tokenizer.decode([101, 7632, 1010, 2129, 2024, 2017, 1029, 102])

'[CLS] hi, how are you? [SEP]'

# Data Pre Processing

In [4]:
class MNLIDataBert(Dataset):

  def __init__(self, train_df, valid_m_df, valid_mm_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

    self.train_df = train_df
    self.valid_m_df = valid_m_df
    self.valid_mm_df = valid_mm_df

    self.base_path = '/content/'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = self.load_data('train', self.train_df)
    self.valid_m_data = self.load_data('valid_m', self.valid_m_df)
    self.valid_mm_data = self.load_data('valid_mm', self.valid_mm_df)

  def load_data(self, data_type, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['premise'].to_list()
    hypothesis_list = df['hypothesis'].to_list()
    label_list = df['label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]

      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(label)

    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(f'The length of {data_type} is {len(dataset)}')
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(self.train_data, shuffle=shuffle, batch_size=batch_size)
    valid_m_loader = DataLoader(self.valid_m_data, shuffle=shuffle, batch_size=batch_size)
    valid_mm_loader = DataLoader(self.valid_mm_data, shuffle=shuffle, batch_size=batch_size)
    return train_loader, valid_m_loader, valid_mm_loader

# Bert for NLI

In [5]:
class BertNLIModel(nn.Module):
    """Performs prediction, given the input of BERT embeddings.
    """
    def __init__(self,model_path=None, gpu=True, bert_type='bert-base', label_num=3, batch_size=32, reinit_num=3, freeze_layers=True):
        super(BertNLIModel, self).__init__()
        self.bert_type = bert_type
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.num_hidden_layers = self.bert.config.num_hidden_layers
        self.vdim = self.bert.config.hidden_size
        self.nli_head = nn.Linear(self.vdim,label_num)
        self.gpu = gpu
        self.batch_size=batch_size
        self.softmax = nn.Softmax(dim=1)
        self.reinit(layer_num=reinit_num, freeze=freeze_layers)

        # load trained model
        if model_path is not None:
            if gpu:
                sdict = torch.load(model_path)
                self.load_state_dict(sdict)
                self.to(device)
            else:
                sdict = torch.load(model_path,map_location=lambda storage, loc: storage)
                self.load_state_dict(sdict)
        else:
            if self.gpu: self.to(device)

    def reinit(self, layer_num, freeze):
        """Reinitialise parameters of last N layers and freeze all others"""
        if freeze:
            for _, pp in self.bert.named_parameters():
                pp.requires_grad = False

        if layer_num >= 0:
            layer_idx = [self.num_hidden_layers-1-i for i in range(layer_num)]
            layer_names = ['encoder.layer.{}'.format(j) for j in layer_idx]
            for pn, pp in self.bert.named_parameters():
                if any([ln in pn for ln in layer_names]) or 'pooler.' in pn:
                    pp.data = torch.randn(pp.shape)*0.02
                    pp.requires_grad = True


    def load_model(self, sdict):
        if self.gpu:
            self.load_state_dict(sdict)
            self.to(device)
        else:
            self.load_state_dict(sdict)

    def forward(self, valid_loader):
        all_probs = None
        labels = []
        # bs = self.batch_size
        for ids_tensor, masks_tensor, types_tensor, label_ground in valid_loader:
            probs = self.ff(ids_tensor, masks_tensor, types_tensor)[1].data.cpu().numpy()
            labels.extend(label_ground.data.cpu().numpy())
            # print(label_ground.data.cpu().numpy())
            if all_probs is None:
                all_probs = probs
            else:
                all_probs = np.append(all_probs, probs, axis=0)

        all_predict = [np.argmax(pp) for pp in all_probs]
        # print('all_probs', all_probs)
        # print('all_predict', all_predict)
        # print('labels', labels)
        assert len(all_predict) == len(labels)

        acc = len([i for i in range(len(labels)) if all_predict[i]==labels[i]])*1./len(labels)

        return acc, all_predict, labels


    def ff(self,ids_tensor, masks_tensor, types_tensor):
      if self.gpu:
        ids_tensor = ids_tensor.to(device)
        types_tensor = types_tensor.to(device)
        masks_tensor = masks_tensor.to(device)

      cls_vecs = self.bert(input_ids=ids_tensor, token_type_ids=types_tensor, attention_mask=masks_tensor)[1]

      logits = self.nli_head(cls_vecs)
      probs = self.softmax(logits)

      # to reduce gpu memory usage
      del ids_tensor
      del types_tensor
      del masks_tensor
      torch.cuda.empty_cache() # releases all unoccupied cached memory

      return logits, probs

    def save(self, output_path, config_dic=None, acc=None):
        if acc is None:
            model_name = 'nli_model.state_dict'
        else:
            model_name = 'nli_model_acc{}.state_dict'.format(acc)
        opath = os.path.join(output_path, model_name)
        if config_dic is None:
            torch.save(self.state_dict(),opath)
        else:
            torch.save(config_dic,opath)

    @staticmethod
    def load(input_path,gpu=True,bert_type='base',label_num=3,batch_size=16):
        if gpu:
            sdict = torch.load(input_path)
        else:
            sdict = torch.load(input_path,map_location=lambda storage, loc: storage)
        model = BertNLIModel(gpu,bert_type,label_num,batch_size)
        model.load_state_dict(sdict)
        return model

# Fine-Tuning

In [9]:
import sys
"""
The system trains BERT on the SNLI + MultiNLI (AllNLI) dataset with softmax loss function.
At every 1000 training steps, the model is evaluated on the dev set.
"""
import logging
from datetime import datetime
import torch.nn as nn
import torch
from tqdm import tqdm
import numpy as np
from transformers import *
import math
import argparse
import random
import copy
import os



def get_scheduler(optimizer, scheduler: str, warmup_steps: int, t_total: int):
    """
    Returns the correct learning rate scheduler
    """
    scheduler = scheduler.lower()
    if scheduler=='constantlr':
        return get_constant_schedule(optimizer)
    elif scheduler=='warmupconstant':
        return get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
    elif scheduler=='warmuplinear':
        return get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
    elif scheduler=='warmupcosine':
        return get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
    elif scheduler=='warmupcosinewithhardrestarts':
        return get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
    else:
        raise ValueError("Unknown scheduler {}".format(scheduler))


def train(model, optimizer, scheduler, train_loader, valid_m_loader, batch_size, gpu, max_grad_norm, best_acc):
    loss_fn = nn.CrossEntropyLoss()

    step_cnt = 0
    best_model_weights = None
    # for pointer in tqdm(range(0, len(train_data), batch_size),desc='training'):
    for ids_tensor, masks_tensor, types_tensor, labels in tqdm(train_loader):
        model.train() # model was in eval mode in evaluate(); re-activate the train mode
        optimizer.zero_grad() # clear gradients first
        torch.cuda.empty_cache() # releases all unoccupied cached memory

        step_cnt += 1

        logits, _ = model.ff(ids_tensor, masks_tensor, types_tensor)
        if logits is None: continue
        true_labels = torch.LongTensor(labels) # ?
        if gpu:
            true_labels = true_labels.to(device)
        loss = loss_fn(logits, true_labels)

        loss.backward()
        train_loss_record.append(loss.detach().item())
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # update weights
        optimizer.step()

        # update training rate
        scheduler.step()

        if step_cnt%200 == 0:
            print('Start evaluation')
            acc = evaluate(model, valid_m_loader)
            print('==> step {} dev acc: {}'.format(step_cnt,acc))
            if acc > best_acc:
                torch.save(model.state_dict(), '/content/MNLI_easy.ckpt')
                best_acc = acc
                best_model_weights = copy.deepcopy(model.cpu().state_dict())
                model.to(device)

    return best_model_weights

def evaluate(model, valid_loader):
  model.eval()
  with torch.no_grad():
      acc, all_predict, all_labels = model(valid_loader)
  prf = precision_recall_fscore_support(all_labels, all_predict, average='macro', labels=[0,1,2])

  print('==>acc<==')
  print(acc)
  print('==>precision-recall-f1<==\n')
  print(prf)

  return acc

def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

def parse_args():
    ap = argparse.ArgumentParser("arguments for bert-nli training")
    ap.add_argument('--batch_size',type=int,default=200,help='batch size')
    ap.add_argument('--epoch_num',type=int,default=5,help='epoch num')
    ap.add_argument('--scheduler_setting',type=str,default='warmupcosinewithhardrestarts',choices=['WarmupLinear','ConstantLR','WarmupConstant','WarmupCosine','WarmupCosineWithHardRestarts'])
    ap.add_argument('--lr',type=int,default=2e-4,help='learning rate')
    ap.add_argument('--eps',type=int,default=1e-6,help='eps')
    ap.add_argument('--trained_model',type=str,default='None',help='path to the trained model; make sure the trained model is consistent with the model you want to train')
    ap.add_argument('--max_grad_norm',type=float,default=1.,help='maximum gradient norm')
    ap.add_argument('--warmup_percent',type=float,default=0.2,help='how many percentage of steps are used for warmup')
    ap.add_argument('--bert_type',type=str,default='bert-base',help='transformer (bert) pre-trained model you want to use', choices=['bert-base','bert-large','albert-base-v2','albert-large-v2'])
    ap.add_argument('--reinit_layers',type=int,default=1,help='reinitialise the last N layers')
    ap.add_argument('--freeze_layers',type=int,default=1,help='whether to freeze all but the lasat few layers (1) or not (0)')

    args = ap.parse_args(args=[])
    return args
args = parse_args()


In [ ]:
batch_size = args.batch_size
epoch_num = args.epoch_num
scheduler_setting = args.scheduler_setting
max_grad_norm = args.max_grad_norm
warmup_percent = args.warmup_percent
bert_type = args.bert_type
trained_model = None if args.trained_model == 'None' else args.trained_model
reinit_layers = args.reinit_layers
freeze_layers = args.freeze_layers
lr = args.lr
eps = args.eps
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu = True if torch.cuda.is_available() else False

print('=====Arguments=====')
print('bert type:\t{}'.format(bert_type))
print('trained model path:\t{}'.format(trained_model))
print('batch size:\t{}'.format(batch_size))
print('epoch num:\t{}'.format(epoch_num))
print('device:\t{}'.format(device))
print('scheduler setting:\t{}'.format(scheduler_setting))
print('max grad norm:\t{}'.format(max_grad_norm))
print('warmup percent:\t{}'.format(warmup_percent))
print('learning rate:\t{}'.format(lr))
print('eps:\t{}'.format(eps))
print('=====Arguments=====')

label_num = 3
train_df = pd.read_csv('/content/mnli-train.csv', usecols = ['label', 'premise', 'hypothesis'])
train_df = train_df.dropna()
valid_m_df = pd.read_csv('/content/mnli-validation_matched.csv', usecols = ['label', 'premise', 'hypothesis'])
valid_m_df = valid_m_df.dropna()
valid_mm_df = pd.read_csv('/content/mnli-validation_mismatched.csv', usecols = ['label', 'premise', 'hypothesis'])
valid_mm_df = valid_mm_df.dropna()

model_save_path = '/content/mnli_{}-{}'.format(bert_type,datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
print('model save path', model_save_path)

MNLIDataset = MNLIDataBert(train_df, valid_m_df, valid_mm_df)
train_loader, valid_m_loader, valid_mm_loader = MNLIDataset.get_data_loaders(batch_size=batch_size)

total_steps = math.ceil(epoch_num*len(MNLIDataset.train_data)*1./batch_size)
warmup_steps = int(total_steps*warmup_percent)

model = BertNLIModel(gpu=gpu, batch_size=batch_size, bert_type=bert_type, model_path=trained_model, reinit_num=reinit_layers, freeze_layers=freeze_layers)
optimizer = AdamW(model.parameters(),lr=lr,eps=eps,correct_bias=False)
scheduler = get_scheduler(optimizer, scheduler_setting, warmup_steps=warmup_steps, t_total=total_steps)

print(f'The model has {count_parameters(model):,} trainable parameters')


In [ ]:
# 使用T4 GPU，batch=200，共训练1 epoch + 1 batch（没GPU了），得到acc=0.742，loss仍然在下降，但无法继续了。
best_acc = -1.
best_model_dic = None
train_loss_record = []
for ep in range(epoch_num):
    print('\n=====epoch {}/{}====='.format(ep,epoch_num))
    model_dic = train(model, optimizer, scheduler, train_loader, valid_m_loader, batch_size, gpu, max_grad_norm, best_acc)
    mean_train_loss = sum(train_loss_record) / len(train_loss_record)
    print(f'current mean loss is {mean_train_loss}')
    if model_dic is not None:
        best_model_dic = model_dic
assert best_model_dic is not None

# for testing load the best model
model.load_model(best_model_dic)
print('\n=====Training finished. Now start test=====')

test_acc = evaluate(model, valid_m_loader)
print('accuracy on test set: {}'.format(test_acc))

if model_save_path is not None:
    os.makedirs(model_save_path, exist_ok=True)
    if os.listdir(model_save_path):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(model_save_path))
model.save(model_save_path,best_model_dic,test_acc)


=====epoch 0/5=====


 10%|█         | 199/1964 [21:12<3:08:16,  6.40s/it]

Start evaluation
==>acc<==
0.48741721854304637
==>precision-recall-f1<==

(0.4874190604043873, 0.4825760466288383, 0.4464833171975419, None)
==> step 200 dev acc: 0.48741721854304637


 20%|██        | 399/1964 [44:45<2:46:43,  6.39s/it]

Start evaluation
==>acc<==
0.6701986754966888
==>precision-recall-f1<==

(0.6686218633695235, 0.6687115237616982, 0.6686185975870611, None)
==> step 400 dev acc: 0.6701986754966888


...

100%|██████████| 1964/1964 [3:51:20<00:00,  7.07s/it]


current mean loss is 0.7610017985955275

=====epoch 1/5=====


 10%|█         | 199/1964 [21:13<3:08:11,  6.40s/it]

Start evaluation
==>acc<==
0.742129393785023
==>precision-recall-f1<==

(0.7412668855950457, 0.7412875459730359, 0.7412055836580996, None)
==> step 200 dev acc: 0.742129393785023


 14%|█▍        | 275/1964 [31:42<2:59:59,  6.39s/it]